In [1]:
import sys

# cd to ../src
sys.path.append("../src")

In [4]:
from model.load_model import load_model_for_inference
import random
import tempfile
import json


import logging
import os


import torch
import torch.nn as nn


from transformers import (
    PreTrainedTokenizerBase,
)
from transformers.modeling_utils import unwrap_model
from transformers.models.auto.modeling_auto import MODEL_FOR_CAUSAL_LM_MAPPING_NAMES

logging.basicConfig(level=logging.INFO)

In [8]:
model, tokenizer = load_model_for_inference(
    weights_path="/gaueko1/hizkuntza-ereduak/LLaMA/lm/huggingface/7B/",
    int8_quantization=True,
    lora_weights_name_or_path="/ikerlariak/osainz006/models/collie/CoLLIE-7b",
)

INFO:root:Device map: auto
INFO:root:Loading model from /gaueko1/hizkuntza-ereduak/LLaMA/lm/huggingface/7B/
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.7/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /ikerlariak/igarcia945/envs/pytorch2/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/ikerlariak/igarcia945/envs/pytorch2/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/lib/boost-1.46.1'), PosixPath('/usr/local/boost/lib')}
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:root:Loading pretrained LORA weights from /ikerlariak/osainz006/models/collie/CoLLIE-7b


In [29]:
prompt = (
    '# The following lines describe the task definition\n@dataclass\nclass Person(Entity):\n    """Persons: first,'
    ' middle and last names of people, animals and fictional\n            characters\n    aliases"""\n\n    span:'
    ' str\n\n\n@dataclass\nclass Organization(Entity):\n    """Organizations: companies (press agencies, studios,'
    " banks, stock\n               markets, manufacturers, cooperatives)\n    subdivisions of companies (newsrooms)\n "
    "   brands\n    political movements (political parties, terrorist\n              organisations)\n    government"
    " bodies (ministries, councils, courts, political unions\n               of countries (e.g. the {\\\\it U.N.}))\n "
    "   publications (magazines, newspapers, journals)\n    musical companies (bands, choirs, opera companies,"
    " orchestras\n    public organisations (schools, universities, charities\n    other collections of people (sports"
    " clubs, sports\n              teams, associations, theaters companies,\n              religious orders, youth"
    ' organisations\n    """\n\n    span: str\n\n\n@dataclass\nclass Location(Entity):\n    """Locations: roads'
    " (streets, motorways)\n    trajectories\n    regions (villages, towns, cities, provinces, countries,"
    " continents,\n             dioceses, parishes)\n    structures (bridges, ports, dams)\n    natural locations"
    " (mountains, mountain ranges, woods,\n                       rivers, wells, fields, valleys, gardens,\n          "
    "             nature reserves, allotments, beaches,\n                       national parks)\n    public places"
    " (squares, opera houses, museums, schools,\n                   markets, airports, stations, swimming pools,\n    "
    "               hospitals, sports facilities, youth centers,\n                   parks, town halls, theaters,"
    " cinemas, galleries,\n                   camping grounds, NASA launch pads, club\n                   houses,"
    " universities, libraries, churches,\n                   medical centers, parking lots, playgrounds,\n            "
    "       cemeteries)\n    commercial places (chemists, pubs, restaurants, depots,\n                       hostels,"
    " hotels, industrial parks,\n                       nightclubs, music venues)\n    assorted buildings (houses,"
    " monasteries, creches, mills,\n                        army barracks, castles, retirement\n                      "
    "  homes, towers, halls, rooms, vicarages,\n                        courtyards)\n    abstract ``places'' (e.g."
    ' {\\\\it the free world})\n    """\n\n    span: str\n\n\n# This is the text to analyze\ntext = "In any event ,'
    ' this question is not presently included among the requests for topical and urgent debate on Thursday ."\n\n# The'
    " annotation instances that take place in the text above are listed here\nresult = ["
)
print(prompt)

# The following lines describe the task definition
@dataclass
class Person(Entity):
    """Persons: first, middle and last names of people, animals and fictional
            characters
    aliases"""

    span: str


@dataclass
class Organization(Entity):
    """Organizations: companies (press agencies, studios, banks, stock
               markets, manufacturers, cooperatives)
    subdivisions of companies (newsrooms)
    brands
    political movements (political parties, terrorist
              organisations)
    government bodies (ministries, councils, courts, political unions
               of countries (e.g. the {\\it U.N.}))
    publications (magazines, newspapers, journals)
    musical companies (bands, choirs, opera companies, orchestras
    public organisations (schools, universities, charities
    other collections of people (sports clubs, sports
              teams, associations, theaters companies,
              religious orders, youth organisations
    """

    span: str




In [30]:
model_input = tokenizer(prompt, add_special_tokens=True, return_tensors="pt")
model_input

{'input_ids': tensor([[    1,   396,   450,  1494,  3454,  8453,   278,  3414,  5023,    13,
         29992,  1272,  1990,    13,  1990,  5196, 29898,  6691,  1125,    13,
          1678,  9995, 15136,   787, 29901,   937, 29892,  7256,   322,  1833,
          2983,   310,  2305, 29892, 15006,   322, 26797,  1848,    13,  9651,
          4890,    13,  1678, 14430,  2129, 15945, 29908,    13,    13,  1678,
         10638, 29901,   851,    13,    13,    13, 29992,  1272,  1990,    13,
          1990,  9205,  2133, 29898,  6691,  1125,    13,  1678,  9995, 27356,
         17063, 29901, 14582,   313,  2139,   946, 15942, 29892,  1921,  2363,
         29892, 24388, 29892, 10961,    13,  1669,  2791,  1691, 29892, 12012,
           332,   414, 29892,  1302,  3372,  5056, 29897,    13,  1678,  1014,
          4563, 12112,   310, 14582,   313, 15753, 18901, 29897,    13,  1678,
          1506,  4167,    13,  1678,  8604, 24147,   313, 20087,   936, 13973,
         29892, 15115,   391,    13,  

In [31]:
model_ouput = model.generate(input_ids=model_input.input_ids.to(model.device), max_new_tokens=32)

In [32]:
print(tokenizer.batch_decode(model_ouput)[0])

<s> # The following lines describe the task definition
@dataclass
class Person(Entity):
    """Persons: first, middle and last names of people, animals and fictional
            characters
    aliases"""

    span: str


@dataclass
class Organization(Entity):
    """Organizations: companies (press agencies, studios, banks, stock
               markets, manufacturers, cooperatives)
    subdivisions of companies (newsrooms)
    brands
    political movements (political parties, terrorist
              organisations)
    government bodies (ministries, councils, courts, political unions
               of countries (e.g. the {\\it U.N.}))
    publications (magazines, newspapers, journals)
    musical companies (bands, choirs, opera companies, orchestras
    public organisations (schools, universities, charities
    other collections of people (sports clubs, sports
              teams, associations, theaters companies,
              religious orders, youth organisations
    """

    span: st

In [34]:
tokenizer.encode("result = []")

[1, 1121, 353, 5159]

In [35]:
tokenizer.encode("result = [")

[1, 1121, 353, 518]

In [36]:
tokenizer.encode("result =[")

[1, 1121, 353, 29961]

In [37]:
tokenizer.encode("result =")

[1, 1121, 353]

In [38]:
tokenizer.encode("result = [")

[1, 1121, 353, 518]

In [54]:
tokenizer.encode("[ ]")

[1, 518, 4514]

In [46]:
tokenizer.encode("8===D")

[1, 29871, 29947, 25512, 29928]

In [52]:
[x for x in tokenizer.vocab if "=" in x]

[':=',
 '▁===',
 '▁=~',
 '=".',
 '="$',
 '▁<=',
 '▁!==',
 '=\\{',
 '<%=',
 '=\\',
 '}=\\',
 "='",
 '▁==>',
 '=`',
 '▁<%=',
 '▁&=\\',
 '={{',
 '>=',
 '"=>',
 "'=>",
 ')=',
 '="/',
 '▁=>',
 '▁=',
 '=/',
 '▁.=',
 '.=',
 '=""',
 '&=',
 '▁+=',
 '=-',
 '+=',
 '<=',
 '=$(',
 '=${',
 '="${',
 '▁-=',
 '%=',
 '====',
 '={',
 '="#',
 '="{',
 ']=',
 '}}=',
 '}=',
 ']="',
 '=="',
 '========',
 '▁>=',
 '▁!=',
 '▁==',
 '=.',
 '▁:=',
 '▁(=',
 '================',
 '!=',
 '===',
 '=>',
 '=&',
 '=(',
 '▁="',
 '▁=\\',
 '="',
 '=[',
 '==',
 '=\\"',
 '▁&=',
 '="@',
 ')=\\',
 '=%',
 '=$',
 '="@+',
 '=',
 '={\\',
 '="<?']